In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.preprocessing import Imputer,StandardScaler,LabelBinarizer,PolynomialFeatures,StandardScaler
from sklearn.pipeline import FeatureUnion,Pipeline
from sklearn.metrics import mean_squared_error
from tensorflow.contrib.layers import fully_connected
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor,Lasso,Ridge,ElasticNet,ElasticNetCV
import tensorflow as tf

In [2]:
raw_dataset=pd.read_csv(r"..\datasets\dsn\train.xls")
y=raw_dataset['Product_Supermarket_Sales']
raw_dataset=raw_dataset.drop(['Supermarket_Identifier','Product_Supermarket_Sales','Product_Identifier','Product_Supermarket_Identifier'],1)

In [3]:
xraw_train,final__x,y_train,final_yl=train_test_split(raw_dataset,y,test_size=.05,random_state=0)
xraw_train,xraw_test,y_train,y_test=train_test_split(xraw_train,y_train,test_size=.2,random_state=0)
xraw_train.sample(3)

,Product_Weight,Product_Fat_Content,Product_Shelf_Visibility,Product_Type,Product_Price,Supermarket_Opening_Year,Supermarket _Size,Supermarket_Location_Type,Supermarket_Type
3292,13.000,Normal Fat,0.083683,Canned,496.36,2009,NaN,Cluster 2,Supermarket Type1
660,7.575,Low Fat,0.055502,Dairy,491.44,2009,NaN,Cluster 2,Supermarket Type1
609,9.695,Normal Fat,0.029159,Canned,436.34,2011,Small,Cluster 2,Supermarket Type1


In [4]:
int_col=['Product_Weight','Product_Shelf_Visibility','Product_Price','Supermarket_Opening_Year']
cat_col=[string for string in list(raw_dataset) if string not in int_col]

class column_picker(BaseEstimator,TransformerMixin):
    def __init__(self,arr,cat=False):
        self.pick=arr
        self.cat=cat
        
    def fit(self,x,y=None):
        return self
        
    def transform(self,x,y=None):
        if self.cat:
            return (x[self.pick].fillna(method='bfill').fillna(method='ffill')).values
        else:
            return np.log(x[self.pick].values+1)
    
    def fit_transform(self,x,y=None):
        if self.cat:
            return pd.get_dummies(x[self.pick].fillna(method='bfill').fillna(method='ffill')).values
        else:
            return np.log(x[self.pick].values+1)
        
def evaluate(pred,actual):
    mse=mean_squared_error(pred,actual)
    return round(np.sqrt(mse),2)

In [5]:
numpipe=Pipeline([("picker",column_picker(int_col)),("imputer",Imputer()),("scaler",StandardScaler())])
catpipe=Pipeline([("picker",column_picker(cat_col,True))])
fullpipe=Pipeline([("feature_combiner",FeatureUnion(transformer_list=[("numpipe",numpipe),("catpipe",catpipe)])),("poly",PolynomialFeatures(degree=2))])
# linear transform
xl_train,xl_test=fullpipe.fit_transform(xraw_train),fullpipe.fit_transform(xraw_test)
final_xl=fullpipe.fit_transform(final__x)
# tree transform
tree_data=pd.get_dummies(raw_dataset).fillna(method="ffill").fillna(method='bfill')
xt_train,final_xt,yt_train,final_yt=train_test_split(tree_data,y,test_size=.05,random_state=0)
xt_train,xt_test,yt_train,yt_test=train_test_split(xt_train,yt_train,test_size=.2,random_state=0)

In [9]:
# numpipe=Pipeline([("picker",column_picker(int_col)),("imputer",Imputer()),("scaler",StandardScaler())])
# lincatpipe=Pipeline([("picker",column_picker(cat_col,True)),("poly",PolynomialFeatures(degree=2))])
# linfullpipe=FeatureUnion(transformer_list=[("numpipe",numpipe),("catpipe",lincatpipe)])
# x_train,x_test=linfullpipe.fit_transform(xraw_train),linfullpipe.fit_transform(xraw_test)
# final_x=linfullpipe.fit_transform(final__x)

False

In [146]:
class superModel():
    
    def __init__(self):
        self.tree_models=[
            RandomForestRegressor(n_estimators=500,max_depth=4),
            RandomForestRegressor(n_estimators=100,max_depth=4),
            RandomForestRegressor(n_estimators=100,max_features=25,max_depth=4),
            RandomForestRegressor(n_estimators=1000,max_depth=3),
            GradientBoostingRegressor(alpha=.5,n_estimators=500,max_depth=3,learning_rate=.0091),
            GradientBoostingRegressor(alpha=.5,n_estimators=1000,max_depth=3,learning_rate=.007),
            GradientBoostingRegressor(alpha=.01,n_estimators=1500,max_depth=3,learning_rate=.006)
        ]
        self.linear_models=[
#             Ridge(alpha=200),
#             ElasticNet(alpha=0.0899999999999999999999999997,max_iter=10000,l1_ratio=.65),
#             Lasso(alpha=10),
        ]
        
    def fit(self,xtree,ytree,xlinear,ylinear):
        ntree=len(self.tree_models)
        nlin=len(self.linear_models)
        for i in range(ntree):
            self.tree_models[i].fit(xtree,ytree)
        for i in range(nlin):
            self.linear_models[i].fit(xlinear,ylinear)
        
        return self
    
    def predict(self,xtree,xlinear):
        ntree=len(self.tree_models)
        nlin=len(self.linear_models)
        n=ntree+nlin
        final=self.tree_models[0].predict(xtree)
        for i in range(1,ntree):
            final+=self.tree_models[i].predict(xtree)
        
        for j in range(nlin):
            final+=self.linear_models[j].predict(xlinear)
        self.predictions=final/n
        return final/n
    
    def evaluation(self,xt=None,xl=None,y=None,former=False):
        if former:
            return evaluate(self.predictions,y)
        else:
            predictions=self.predict(xt,xl)
            return evaluate(predictions,y)

In [91]:
combined_model=superModel().fit(xt_train,yt_train,xl_train,y_train)

In [92]:
combined_model.evaluation(xt_test,xl_test,yt_test)

2853.0599999999999

In [93]:
combined_model.evaluation(final_xt,final_xl,final_yt)

3048.0

In [39]:
final_yl[:4]

3099    9494.31
2731    5970.56
4593    5667.62
376     6055.45
Name: Product_Supermarket_Sales, dtype: float64

In [ ]:
#########################finaltest###############################################################################################


In [44]:
final_yl.shape

(250,)

In [ ]:
################################################################################################################################
########################################pre#####################################################################################

In [43]:
from sklearn.linear_model import ElasticNet as Model
model=Model(alpha=0.0899999999999999999999999997,max_iter=10000,l1_ratio=.65).fit(xl_train,y_train)
pred=model.predict(xl_train)
t_pred=model.predict(final_xl)
print("rmse->train:{} test:{}".format(evaluate(pred,y_train),evaluate(t_pred,final_yl)))
print("r-score->train{} test:{}".format(model.score(xl_train,y_train),model.score(xl_test,y_test)))
# 2985 20
# yes

rmse->train:2872.28 test:3072.81
r-score->train0.5846706443869527 test:0.5704713609036767


In [45]:
# final model

from sklearn.linear_model import Lasso as Model
model=Model(alpha=24).fit(xl_train,y_train)
pred=model.predict(xl_train)
t_pred=model.predict(final_xl)
print("rmse->train:{} test:{}".format(evaluate(pred,y_train),evaluate(t_pred,final_yl)))
print("r-score->train{} test:{}".format(model.score(xl_train,y_train),model.score(xl_test,y_test)))
# 2975
# yes

rmse->train:2907.47 test:3060.38
r-score->train0.5744317932191785 test:0.5774887147150114


In [143]:
# final model

from sklearn.linear_model import SGDRegressor as Model
model=Model(max_iter=1000).fit(xl_train,y_train)
pred=model.predict(xl_train)
t_pred=model.predict(xl_test)
print("rmse->train:{} test:{}".format(evaluate(pred,y_train),evaluate(t_pred,y_test)))
print("r-score->train{} test:{}".format(model.score(xl_train,y_train),model.score(xl_test,y_test)))
# 2975
# yes

rmse->train:2822.7 test:2934.98
r-score->train0.5988871141425729 test:0.5525737797567668


In [145]:
# final model

from sklearn.linear_model import Lasso as Model
model=Model(alpha=50).fit(xl_train,y_train)
pred=model.predict(xl_train)
t_pred=model.predict(xl_test)
print("rmse->train:{} test:{}".format(evaluate(pred,y_train),evaluate(t_pred,y_test)))
print("r-score->train{} test:{}".format(model.score(xl_train,y_train),model.score(xl_test,y_test)))
# 2975
# yes

rmse->train:2926.88 test:2865.91
r-score->train0.5687313049285233 test:0.5733859043678944


In [21]:
from sklearn.ensemble import RandomForestRegressor as Model
model=Model(n_estimators=1000,max_depth=3).fit(xt_train,yt_train)
pred=model.predict(xt_train)
t_pred=model.predict(xt_test)
print("rmse->train:{} test:{}".format(evaluate(pred,yt_train),evaluate(t_pred,yt_test)))
print("r-score->train{} test:{}".format(model.score(xt_train,yt_train),model.score(xt_test,yt_test)))
# hell yes

rmse->train:3010.33 test:2953.59
r-score->train0.5437879544951464 test:0.5468823291898104


In [165]:
from sklearn.ensemble import RandomForestRegressor as Model
model=Model(n_estimators=100,max_depth=4).fit(xt_train,yt_train)
pred=model.predict(xt_train)
t_pred=model.predict(xt_test)
print("rmse->train:{} test:{}".format(evaluate(pred,yt_train),evaluate(t_pred,yt_test)))
print("r-score->train{} test:{}".format(model.score(xt_train,yt_train),model.score(xt_test,yt_test)))
# 2948
# hell yes

rmse->train:2897.86 test:2876.95
r-score->train0.5930665897561566 test:0.5364837603155334


In [20]:
from sklearn.ensemble import RandomForestRegressor as Model
model=Model(n_estimators=100,max_features=25,max_depth=4).fit(xt_train,yt_train)
pred=model.predict(xt_train)
t_pred=model.predict(xt_test)
print("rmse->train:{} test:{}".format(evaluate(pred,yt_train),evaluate(t_pred,yt_test)))
print("r-score->train{} test:{}".format(model.score(xt_train,yt_train),model.score(xt_test,yt_test)))
# 2948
# hell yes

rmse->train:2894.81 test:2870.09
r-score->train0.5781317319245332 test:0.5721403053559833


In [170]:
from sklearn.ensemble import RandomForestRegressor as Model
model=Model(n_estimators=1000,max_depth=4).fit(xt_train,yt_train)
pred=model.predict(xt_train)
t_pred=model.predict(xt_test)
print("rmse->train:{} test:{}".format(evaluate(pred,yt_train),evaluate(t_pred,yt_test)))
print("r-score->train{} test:{}".format(model.score(xt_train,yt_train),model.score(xt_test,yt_test)))
# 2948
# hell yes

rmse->train:2896.34 test:2871.88
r-score->train0.593492468277109 test:0.5381177887751254


In [73]:
from sklearn.ensemble import GradientBoostingRegressor as Model
model=Model(alpha=.5,n_estimators=500,max_depth=2,learning_rate=.0091).fit(xt_train,yt_train)
pred=model.predict(xt_train)
t_pred=model.predict(xt_test)
print("rmse->train:{} test:{}".format(evaluate(pred,yt_train),evaluate(t_pred,yt_test)))
print("r-score->train{} test:{}".format(model.score(xt_train,yt_train),model.score(xt_test,yt_test)))

rmse->train:2893.06 test:2883.95
r-score->train0.5786415092970274 test:0.5679995717099718


In [22]:
from sklearn.ensemble import GradientBoostingRegressor as Model
model=Model(alpha=.5,n_estimators=1000,max_depth=3,learning_rate=.007).fit(xt_train,yt_train)
pred=model.predict(xt_train)
t_pred=model.predict(xt_test)
print("rmse->train:{} test:{}".format(evaluate(pred,yt_train),evaluate(t_pred,yt_test)))
print("r-score->train{} test:{}".format(model.score(xt_train,yt_train),model.score(xt_test,yt_test)))

rmse->train:2760.49 test:2871.47
r-score->train0.6163710572015624 test:0.5717279201830983


In [25]:
from sklearn.ensemble import GradientBoostingRegressor as Model
model=Model(alpha=.01,n_estimators=1500,max_depth=3,learning_rate=.006).fit(xt_train,yt_train)
pred=model.predict(xt_train)
t_pred=model.predict(xt_test)
print("rmse->train:{} test:{}".format(evaluate(pred,yt_train),evaluate(t_pred,yt_test)))
print("r-score->train{} test:{}".format(model.score(xt_train,yt_train),model.score(xt_test,yt_test)))

rmse->train:2840.7 test:2877.69
r-score->train0.5937549105778037 test:0.5698716806154065


In [ ]:
################################################################################################################################
##############################################################submission########################################################
################################################################################################################################
################################################################################################################################

In [94]:
final=pd.read_csv(r"../datasets/dsn/test.xls")
final=final.drop(['Supermarket_Identifier','Product_Identifier','Product_Supermarket_Identifier'],1)

final_xl=fullpipe.fit_transform(final)
final_xt=pd.get_dummies(final).fillna(method="ffill").fillna(method='bfill')

fullxl=fullpipe.fit_transform(raw_dataset)
fullxt=pd.get_dummies(raw_dataset).fillna(method="ffill").fillna(method='bfill')

In [95]:
final.head()

,Product_Weight,Product_Fat_Content,Product_Shelf_Visibility,Product_Type,Product_Price,Supermarket_Opening_Year,Supermarket _Size,Supermarket_Location_Type,Supermarket_Type
0,8.270,Normal Fat,0.214125,Soft Drinks,459.98,2005,NaN,Cluster 3,Grocery Store
1,8.270,Normal Fat,0.127821,Soft Drinks,464.98,1994,High,Cluster 3,Supermarket Type1
2,7.390,Low Fat,0.082171,Soft Drinks,477.38,1994,High,Cluster 3,Supermarket Type1
3,6.115,Normal Fat,0.011791,Soft Drinks,472.63,2005,NaN,Cluster 3,Grocery Store
4,6.115,Normal Fat,0.007038,Soft Drinks,473.13,1994,High,Cluster 3,Supermarket Type1


In [147]:
model=superModel().fit(fullxt,y,fullxl,y)
predictions=model.predict(final_xt,final_xl)

In [153]:
# output
form=pd.read_csv(r"dsn/final.csv")
# form["Product_Supermarket_Sales"]=predictions
form["Product_Supermarket_Sales"]=data
form.to_csv(r"com_allalltree_straw.csv")

In [149]:
predictions

array([ 1234.209215  ,  8092.97419088,  8090.44757348, ...,  9332.08151538,
        1395.19099919,  8081.26035279])

In [150]:
one=pd.read_csv(r"fall.csv")
one["Product_Supermarket_Sales"].values

array([ 1423.095938,  7853.438402,  7948.840838, ...,  9955.984713,
        1509.035899,  7667.560609])

In [151]:
data=(one["Product_Supermarket_Sales"]+predictions)/2

In [152]:
data

0       1328.652577
1       7973.206296
2       8019.644206
3       1416.838119
4       7928.669678
5       4374.805974
6       7918.548389
7       4355.166184
8       2677.605973
9       6754.372126
10      2761.570249
11      2624.101863
12      2589.041166
13      2715.426180
14      2503.240886
15      6654.945796
16      2756.995420
17      2621.506754
18      5284.073954
19      5305.111750
20      3360.611384
21       363.509634
22      7705.591677
23      3629.455786
24      3532.204254
25      1800.913816
26      9048.094337
27      9541.013934
28      7509.919347
29      2817.716498
           ...     
3502    2678.312786
3503    2750.960068
3504    2563.588978
3505    2585.228865
3506    4395.427741
3507    7994.541180
3508     710.751916
3509    4305.756704
3510    9861.182556
3511    9182.914770
3512    9437.499987
3513    9722.924715
3514    2744.078004
3515    2460.263385
3516    6464.529510
3517    2680.656816
3518    2617.337849
3519    8976.293805
3520    9457.757251


In [41]:
tf.contrib.framework.arg_scope

<function tensorflow.contrib.framework.python.ops.arg_scope.arg_scope>

In [6]:
dsn=tf.Graph()
n=xl_train.shape[1]
with dsn.as_default():
    x=tf.placeholder(tf.float32,shape=(None,n))
    y=tf.placeholder(tf.float32,shape=(None,1))
    drop=tf.placeholder(tf.float32)
    with tf.contrib.framework.arg_scope(
        [fully_connected],
        weights_regularizer=tf.contrib.layers.l1_regularizer(scale=0.05)
    ):
        hidden1=tf.nn.dropout(fully_connected(x,200,activation_fn=tf.nn.relu),drop)
#         hidden2=tf.nn.dropout(fully_connected(hidden1,500,activation_fn=tf.nn.elu),drop)
#         hidden3=tf.nn.dropout(fully_connected(hidden2,50,activation_fn=tf.nn.relu),drop)
        pred=fully_connected(hidden1,1,activation_fn=None)
    base_loss=tf.sqrt(tf.reduce_mean(tf.square(pred-y)))
    reg_loss=tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss=tf.add_n([base_loss]+reg_loss,name='loss')
    train_op=tf.train.AdamOptimizer(0.001).minimize(loss)
    saver=tf.train.Saver()

In [ ]:
n_epoch=100000
with dsn.as_default():
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
#         saver.restore(sess,r"/temp/tf_models/dsn.ckpt")
        initial_loss=sess.run([loss],{x:xl_test,y:y_test.values.reshape(-1,1),drop:1})
        print("initial loss:{}".format(initial_loss))
        for epoch in range(n_epoch):
            _,train_loss=sess.run([train_op,loss],{x:xl_train,y:y_train.values.reshape(-1,1),drop:.85})
            test_loss=sess.run([loss],{x:xl_test,y:y_test.values.reshape(-1,1),drop:1})
            if epoch%100==0:
                saver.save(sess,r"/temp/tf_models/dsn.ckpt")
                print("epoch:{} train_loss:{} test_loss:{}".format(epoch,train_loss,*test_loss))
    

initial loss:[7763.4712]
epoch:0 train_loss:7818.29150390625 test_loss:7757.55322265625
epoch:100 train_loss:7552.37451171875 test_loss:7497.11865234375
epoch:200 train_loss:7499.16259765625 test_loss:7442.216796875
epoch:300 train_loss:7355.71826171875 test_loss:7295.59912109375
epoch:400 train_loss:7099.6533203125 test_loss:7034.8134765625
epoch:500 train_loss:6735.9501953125 test_loss:6665.10791015625
epoch:600 train_loss:6277.5908203125 test_loss:6203.8837890625
epoch:700 train_loss:5762.90673828125 test_loss:5685.49072265625
epoch:800 train_loss:5246.23974609375 test_loss:5161.994140625
epoch:900 train_loss:4791.236328125 test_loss:4693.900390625
epoch:1000 train_loss:4424.4501953125 test_loss:4327.16943359375
epoch:1100 train_loss:4165.05419921875 test_loss:4071.865966796875
epoch:1200 train_loss:4007.99072265625 test_loss:3906.561767578125
epoch:1300 train_loss:3885.513671875 test_loss:3797.345947265625
epoch:1400 train_loss:3810.29931640625 test_loss:3719.79736328125
epoch:1500